In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

### Reading first dataset

In [30]:
amazon_ratings = pd.read_csv('data/Amazon/Books_rating.csv')
amazon_books_data = pd.read_csv('data/Amazon/books_data.csv')

In [31]:
amazon_ratings.head(3)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."


In [32]:
amazon_books_data.head(3)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN


### Reading second dataset

In [35]:
def clean_string(s):
    if s is not np.nan:
        cleaned = ''.join(char for char in s.lower() if char.isalnum() and (char.isascii() or char.isdigit()))
        return cleaned
    return np.nan

In [38]:
books = pd.read_csv('data/Second dataset/Books.csv')
ratings = pd.read_csv('data/Second dataset/Ratings.csv')
users = pd.read_csv('data/Second dataset/Users.csv')

C:\Users\just_\AppData\Local\Temp\ipykernel_12548\716747401.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/Second dataset/Books.csv')


In [43]:
merged_data = ratings.merge(books, on='ISBN').merge(users, on='User-ID')
merged_data['raw_title'] = merged_data['Book-Title'].apply(clean_string)
merged_data['raw_year'] = pd.to_datetime(merged_data['Year-Of-Publication'], format='mixed', errors='coerce').dt.year

In [44]:
merged_data.head(3)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age,raw_title,raw_year
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN,fleshtonesanovel,1970.0
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0,fleshtonesanovel,1970.0
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,"cincinnati, ohio, usa",23.0,endersgameenderwigginssagapaperback,1970.0


### Merging all data

- Books can be merged on Title, publisher and Year of publication
- We will use Amazon rewies for our summarization model
- We will use our merged dataset based on second dataset for recommendation system

In [54]:
amazon_books_data = amazon_books_data.dropna(subset=['Title'])
amazon_books_data['raw_title'] = amazon_books_data['Title'].apply(clean_string)
amazon_books_data['raw_year'] = pd.to_datetime(amazon_books_data['publishedDate'].apply(clean_string), format='mixed', errors='coerce')
amazon_books_data['raw_year'] = pd.to_datetime(amazon_books_data['raw_year'], utc=True).dt.year

In [62]:
all_data_merged = merged_data.merge(amazon_books_data[['raw_title', 'categories', 'description']], how='left', on=['raw_title'])